In [2]:
!pip install datasets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.5 MB/s eta 0:00:00


In [7]:
from huggingface_hub import notebook_login
notebook_login()


In [8]:
# Extract the Lima dataset’s instruction
from datasets import load_dataset

lima = load_dataset("GAIR/lima")

lima.py:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/27.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [10]:
import json
parsed_conversations = [
    json.loads(conv) if isinstance(conv, str) else conv
    for conv in lima["train"]["conversations"]
]

In [12]:
instructions = []
for conv in parsed_conversations:
    try:
        instructions.append(conv[0]["content"])
    except (KeyError, IndexError, TypeError):
        instructions.append("")  # 跳过无效条目或标记为空

In [13]:
#Sample 50 instructions
from datasets import Dataset
sampled_dataset = Dataset.from_dict({"instruction": instructions}).shuffle(seed=42).select(range(50))

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [15]:
# use mistralai/Mistral-7B-Instruct-v0.2 to generate 5 responses for each instruction
import torch
from tqdm import tqdm

all_responses = []
for instruction in tqdm(sampled_dataset["instruction"]):
    inputs = tokenizer.apply_chat_template(
        [{"role": "user", "content": instruction}],
        return_tensors="pt"
    ).to(model.device)

    # 生成5个不同回答
    outputs = model.generate(
        inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=5,
        do_sample=True
    )
    responses = [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]
    all_responses.append(responses)

  0%|          | 0/50 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
  0%|          | 0/50 [16:44<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-376a1e8ecdc8>", line 13, in <cell line: 0>
    outputs = model.generate(
              ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py", line 2255, in generate
    result = self._sample(
             ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py", line 3257, in _sample
    outputs = model_forward(**model_inputs, return_dict=True)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    

TypeError: object of type 'NoneType' has no len()

In [ ]:
#use PairRM to create a preference dataset
from pairrm import PairRMScorer

scorer = PairRMScorer()
preference_data = []

for idx, instruction in tqdm(enumerate(sampled_dataset["instruction"])):
    candidates = all_responses[idx]
    pairs = [(i, j) for i in range(5) for j in range(5) if i < j]
    scores = []

    for i, j in pairs:
        score = scorer.score([instruction], [candidates[i]], [candidates[j]])
        scores.append((i, j, score))

    best_idx = max(scores, key=lambda x: x[2])[0]
    worst_idx = min(scores, key=lambda x: x[2])[1]
    preference_data.append({
        "instruction": instruction,
        "chosen": candidates[best_idx],
        "rejected": candidates[worst_idx],
    })

from datasets import Dataset

dataset = Dataset.from_dict({
    "instruction": [d["instruction"] for d in preference_data],
    "chosen": [d["chosen"] for d in preference_data],
    "rejected": [d["rejected"] for d in preference_data],
})
# Push this dataset to huggingface
dataset.push_to_hub("ShuyanCHEN/DSAA6000_assignment4")

In [ ]:
#Use DPO to fine tune mistralai/Mistral-7B-Instruct-v0.2
from transformers import TrainingArguments
from trl import DPOTrainer
import pandas as pd
import torch

#sample 10 instructions that were not seen in training and generate samples
dpo_dataset = dataset.train_test_split(
    test_size=0.2,
    shuffle=True,
    seed=42
)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=5e-6,
    num_train_epochs=2,
    logging_steps=10,
    evaluation_strategy="no",
    output_dir="./dpo_model",
    fp16=True,
    optim="adamw_torch",
    report_to="none"
)

In [ ]:
#Compare the completions from the original model (mistralai/Mistral-7B-Instruct-v0.2 and your DPO fine tuned model
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=training_args,
    train_dataset=dpo_dataset["train"],
    tokenizer=tokenizer,
    beta=0.1,
    max_length=512,
)
dpo_trainer.train()

In [ ]:

test_instructions = dpo_dataset["test"]["instruction"][:10]

original_outputs = []
dpo_outputs = []


def generate_response(instruction, model):
    inputs = tokenizer.apply_chat_template(
        [{"role": "user", "content": instruction}],
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


for instr in test_instructions:

    original_model = AutoModelForCausalLM.from_pretrained(
        "mistralai/Mistral-7B-Instruct-v0.2",
        device_map="auto"
    )
    original_outputs.append(generate_response(instr, original_model))


    dpo_outputs.append(generate_response(instr, dpo_trainer.model))


In [ ]:
# Display the instruction, original model completion, and DPO fine-tuned model completion as a pandas dataframe
df = pd.DataFrame({
    "Instruction": test_instructions,
    "Original Model": original_outputs,
    "DPO Model": dpo_outputs,
})

pd.set_option("display.max_colwidth", 200)
pd.set_option("display.width", 1000)

#print out the dataframe to stdout
print(df.to_markdown(index=False))



In [ ]:
#Push the PEFT adapter to huggingface
dpo_trainer.model.save_pretrained("mistral-7b-dpo-adapter")

from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="mistral-7b-dpo-adapter",
    repo_id="ShuyanCHEN/DSAA6000_assignment4",
    repo_type="model"
)

print("\nhttps://huggingface.co/ShuyanCHEN/DSAA6000_assignment4")
